# Pair Programming 30/05 Regresión lineal múltiple II - Tatiana y Guada

In [1]:
# Importando paquetes! 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings( "ignore", module = "seaborn\..*" )
import sidetable
import statsmodels.api as sm
import pylab as py
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ejercicio 1
Tomando los datos del mejor modelo obtenido en la clase de la pair de ayer.
Investiga la multicolinealidad de las variables independientes.
Selecciona aquellas variables que tengan una colinealidad dentro de los margenes estandar. Mostrando como se han ido eliminando las variables y como han cambiado los resultados del VIF.
Realiza un modelo de regresión lineal multiple para ver si se han mejorado los resultados obtenidos.
Estudia si tus resultados son acordes a las asunciones de linealidad.

In [9]:
df= pd.read_csv('df_elegido.csv', index_col=0) 
df.head(2)

,age,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,...,salary,x0_ ?,x0_ Federal-gov,x0_ Local-gov,x0_ Never-worked,x0_ Private,x0_ Self-emp-inc,x0_ Self-emp-not-inc,x0_ State-gov,x0_ Without-pay
0,39,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,38,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
df_numericas = df.select_dtypes(include=np.number)

In [4]:
def get_VIF(dataframe, target):
        X = add_constant(dataframe.loc[:, dataframe.columns != target])
        seriesObject = pd.Series([variance_inflation_factor(X.values,i) for i in range(X.shape[1])] , index=X.columns,).sort_values(ascending= False)
        return seriesObject

In [5]:
get_VIF(df_numericas,'capital-gain')

/home/guadalupec/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/guadalupec/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/guadalupec/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/guadalupec/anaconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


x0_ ?                        inf
x0_ Federal-gov              inf
x0_ Local-gov                inf
x0_ Never-worked             inf
x0_ Private                  inf
x0_ Self-emp-inc             inf
x0_ Self-emp-not-inc         inf
x0_ State-gov                inf
x0_ Without-pay              inf
education-num           1.064752
hours-per-week          1.047127
age                     1.043744
salary                  1.000283
const                   0.000000
capital-loss                 NaN
dtype: float64

Observamos que ninguna de nuestras variables tienen colinealidad, por lo que no eliminaremos ninguna.

In [12]:
x= df_numericas[['age', 'education-num', 'capital-loss',"hours-per-week"]] #independientes
y = df_numericas['capital-gain']#dependientes
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Creamos la regresiónn lineal
LR = LinearRegression( normalize=True) # nos aseguramos de que se normalizan los datos
# Ajustamos el modelo
LR.fit(x_train,y_train)
y_prediction_train =  LR.predict(x_train)
Adj_r2_train = 1 - (1-r2_score(y_train,y_prediction_train)) * (x_train.shape[0]-1)/(x_train.shape[0]-x_train.shape[1]-1)

y_prediction =  LR.predict(x_test)
Adj_r2_test = 1 - (1-r2_score(y_test, y_prediction)) * (x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)

# Para el conjunto de entrenamiento las méticas han sido
print('Para el conjunto train:---------------')
print('El valor de r2 score es ',r2_score(y_train,y_prediction_train))
print('El valor de r2 score adjusted es ',Adj_r2_train)
print('El MAE es',mean_absolute_error(y_train,y_prediction_train))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_train,y_prediction_train)))


# Para el conjunto de test las métricas han sido
print('Para el conjunto test:---------------')
print('El valor de r2 score es ',r2_score(y_test,y_prediction))
print('El valor de r2 score adjusted es ',Adj_r2_test)
print('El MAE es',mean_absolute_error(y_test,y_prediction))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_test,y_prediction)))

Para el conjunto train:---------------
El valor de r2 score es  0.021958715154813624
El valor de r2 score adjusted es  0.021731857216896522
El MAE es 1915.0642301486994
EL RMSE es  6706.3449530970975
Para el conjunto test:---------------
El valor de r2 score es  0.02380552999892227
El valor de r2 score adjusted es  0.022899128448317874
El MAE es 2089.3566415672267
EL RMSE es  7739.030224808847


# Ejercicio 2
Toma ahora el dataset del mejor modelo obtenido y aplica los siguentes modelos de regresión con regularización.
Ridge
Lasso.

In [10]:
## Ridge

x= df_numericas.drop(columns = ['capital-gain'])
y = df_numericas['capital-gain']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Creamos la regresiónn lineal
LR = Ridge(normalize= True) # nos aseguramos de que se normalizan los datos
# Ajustamos el modelo

LR.fit(x_train,y_train)
y_prediction_train =  LR.predict(x_train)
Adj_r2_train = 1 - (1-r2_score(y_train,y_prediction_train)) * (x_train.shape[0]-1)/(x_train.shape[0]-x_train.shape[1]-1)

y_prediction =  LR.predict(x_test)
Adj_r2_test = 1 - (1-r2_score(y_test, y_prediction)) * (x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)

# Para el conjunto de entrenamiento las méticas han sido
print('Para el conjunto train:---------------')
print('El valor de r2 score es ',r2_score(y_train,y_prediction_train))
print('El valor de r2 score adjusted es ',Adj_r2_train)
print('El MAE es',mean_absolute_error(y_train,y_prediction_train))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_train,y_prediction_train)))


# Para el conjunto de test las métricas han sido
print('Para el conjunto test:---------------')
print('El valor de r2 score es ',r2_score(y_test,y_prediction))
print('El valor de r2 score adjusted es ',Adj_r2_test)
print('El MAE es',mean_absolute_error(y_test,y_prediction))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_test,y_prediction)))

Para el conjunto train:---------------
El valor de r2 score es  0.022598105218575304
El valor de r2 score adjusted es  0.021804161120696608
El MAE es 1842.246975764632
EL RMSE es  6704.1524733761025
Para el conjunto test:---------------
El valor de r2 score es  0.021572212804834567
El valor de r2 score adjusted es  0.018385151608759087
El MAE es 1997.676208314222
EL RMSE es  7747.877762754579


In [11]:
#Lasso

x=df_numericas.drop(columns = ['capital-gain'])
y = df_numericas['capital-gain']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Creamos la regresiónn lineal
LR = Lasso(normalize= True) # nos aseguramos de que se normalizan los datos
# Ajustamos el modelo

LR.fit(x_train,y_train)
y_prediction_train =  LR.predict(x_train)
Adj_r2_train = 1 - (1-r2_score(y_train,y_prediction_train)) * (x_train.shape[0]-1)/(x_train.shape[0]-x_train.shape[1]-1)

y_prediction =  LR.predict(x_test)
Adj_r2_test = 1 - (1-r2_score(y_test, y_prediction)) * (x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)

# Para el conjunto de entrenamiento las méticas han sido
print('Para el conjunto train:---------------')
print('El valor de r2 score es ',r2_score(y_train,y_prediction_train))
print('El valor de r2 score adjusted es ',Adj_r2_train)
print('El MAE es',mean_absolute_error(y_train,y_prediction_train))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_train,y_prediction_train)))


# Para el conjunto de test las métricas han sido
print('Para el conjunto test:---------------')
print('El valor de r2 score es ',r2_score(y_test,y_prediction))
print('El valor de r2 score adjusted es ',Adj_r2_test)
print('El MAE es',mean_absolute_error(y_test,y_prediction))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_test,y_prediction)))

Para el conjunto train:---------------
El valor de r2 score es  0.02619413039586227
El valor de r2 score adjusted es  0.025403107351217202
El MAE es 1858.2179652709713
EL RMSE es  6691.808259230228
Para el conjunto test:---------------
El valor de r2 score es  0.024051587744290392
El valor de r2 score adjusted es  0.020872602688082997
El MAE es 2022.1597733102392
EL RMSE es  7738.054820625862


El valor de r2 score es  0.022598105218575304
El valor de r2 score adjusted es  0.021804161120696608
El MAE es 1842.246975764632
EL RMSE es  6704.1524733761025
Para el conjunto train:---------------
El valor de r2 score es  0.02619413039586227
El valor de r2 score adjusted es  0.025403107351217202
El MAE es 1858.2179652709713
EL RMSE es  6691.808259230228


# Ejercicio 3
Compara los resultados del ajuste lineal multiple donde se han eliminado variables y los obtenidos aplicando las regresiones de tipo Ridge y Lassso. Explica los resultados obtenidos.

Tanto para las regresiones con penalizacion de tipo Ridge y Lasso, nuestras variaciones, aunque han sido menores, son insignificantes para poder hacer un análisis sobre ello. 

El valor de r2 en nuestros 3 modelos no cambia significativamente, pero si vemos cambios importantes en el MAE inicial y el modelo de Lasso. Sin embargo, vemos que nuestros modelos no responden a una regresión lineal, por lo que se debe intentar buscar una regresión logarítmica.